# Phase 1.4 — Knowledge Ingestion into ChromaDB

**Objective:** Ingest the verified Phase 1.2 vectors into ChromaDB to build the RAG Knowledge Base,
while strictly reserving a 20% "Live Fire" dataset for future testing.

---

## Pipeline Overview

```
X_knowledge_vectors_v4.npy  (9,152,557 × 50)
X_knowledge_meta_v4.parquet (9,152,557 × 8)
         │
         ▼
┌─────────────────────────────────────────────┐
│        Stratified Split  (sklearn)          │
│   80% Ingest Set  |  20% Live Set           │
└──────────┬──────────────────────┬──────────┘
           │                      │
           ▼                      ▼
    ChromaDB Ingest        live_simulation_data_v4.npy
    (rag_ids_knowledge_base)   live_simulation_meta_v4.parquet
    cosine / hnsw
```

| Step | Action | Output |
|------|--------|--------|
| 1 | Stratified 80/20 split | Ingest Set + Live Set arrays |
| 2 | Save Live Set | `live_simulation_data_v4.npy`, `live_simulation_meta_v4.parquet` |
| 3 | Verify split disjoint | Assertion pass |
| 4 | Init ChromaDB | Persistent collection at `chromadb_store/` |
| 5 | Batch ingest (40 K/batch) | `rag_ids_knowledge_base` collection |
| 6 | Sanity-check query | Nearest-neighbour from Live Set |

In [5]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 1 | Imports & Path Configuration
# ─────────────────────────────────────────────────────────────────────────────
import sys, os, time, warnings
from pathlib import Path

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=FutureWarning)

try:
    import chromadb
    print(f"✅ chromadb       {chromadb.__version__}")
except ImportError:
    raise SystemExit("pip install chromadb")

try:
    from sklearn.model_selection import train_test_split
    import sklearn
    print(f"✅ scikit-learn   {sklearn.__version__}")
except ImportError:
    raise SystemExit("pip install scikit-learn")

try:
    from tqdm.auto import tqdm
    print(f"✅ tqdm")
except ImportError:
    raise SystemExit("pip install tqdm")

print(f"✅ numpy          {np.__version__}")
print(f"✅ pandas         {pd.__version__}")
print(f"✅ Python         {sys.version.split()[0]}")

# ── Paths ─────────────────────────────────────────────────────────────────────
NOTEBOOK_DIR  = Path.cwd()
MAIN_DIR      = NOTEBOOK_DIR.parent
PROCESSED_DIR = MAIN_DIR / "data" / "processed"
CHROMA_DIR    = MAIN_DIR / "chromadb_store_v4"

VECTORS_PATH          = PROCESSED_DIR / "X_knowledge_vectors_v4.npy"
META_PATH             = PROCESSED_DIR / "X_knowledge_meta_v4.parquet"
LIVE_VECTORS_PATH     = PROCESSED_DIR / "live_simulation_data_v4.npy"
LIVE_META_PATH        = PROCESSED_DIR / "live_simulation_meta_v4.parquet"

CHROMA_DIR.mkdir(parents=True, exist_ok=True)

# ── Constants ─────────────────────────────────────────────────────────────────
COLLECTION_NAME  = "rag_ids_knowledge_base"
# ChromaDB hard limit is 5461 per batch — use 5000 to stay safely under it
BATCH_SIZE       = 5_000
TEST_SIZE        = 0.20      # 20% live simulation
RANDOM_STATE     = 42
LABEL_COL        = "univ_label_multiclass"

LABEL_NAMES = {
    0: "Normal",
    1: "Reconnaissance",
    2: "Volumetric_Attack",
    3: "C2_Botnet",
    4: "Exploit_and_Theft",
}

print(f"\n📁 Path verification:")
for name, p in [
    ("VECTORS",    VECTORS_PATH),
    ("META",       META_PATH),
    ("CHROMA_DIR", CHROMA_DIR),
]:
    print(f"   {'✅' if p.exists() else '❌'} {name:<12} {p}")

✅ chromadb       1.4.1
✅ scikit-learn   1.7.0
✅ tqdm
✅ numpy          2.1.3
✅ pandas         2.2.3
✅ Python         3.13.9

📁 Path verification:
   ✅ VECTORS      c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\data\processed\X_knowledge_vectors_v4.npy
   ✅ META         c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\data\processed\X_knowledge_meta_v4.parquet
   ✅ CHROMA_DIR   c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\chromadb_store_v4


---
## Environment Reset — Clean ChromaDB Build for v4

**Objective:** Delete the existing ChromaDB store (built from v2 vectors - 6.75M vectors) to ensure we build a clean v4 knowledge base from **9.15M v4 vectors**.

The existing `chromadb_store/` directory must be removed before ingesting the new v4 vectors to prevent contamination or version conflicts.

**Note:** For this v4 rebuild, we will use a new directory `chromadb_store_v4/` to avoid file lock conflicts with the existing v2 store.

In [7]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 1.5 | Delete Existing ChromaDB Store
# ─────────────────────────────────────────────────────────────────────────────
import shutil

print("🗑️  Checking for existing ChromaDB v4 store …")

# Using chromadb_store_v4 to avoid file locks from v2 store
if CHROMA_DIR.exists():
    total_size = sum(f.stat().st_size for f in CHROMA_DIR.rglob("*") if f.is_file())
    print(f"   Found: {CHROMA_DIR}  ({total_size / 1e9:.3f} GB)")
    try:
        shutil.rmtree(CHROMA_DIR)
        print(f"   ✅ Deleted — clean slate for v4")
    except PermissionError as e:
        print(f"   ⚠️  Unable to delete (file locks): {e}")
        print(f"   ℹ️  Will create new directory - collection delete in Step 5 will ensure clean state")
else:
    print(f"   ⏭️  No existing v4 store found — clean start")

# Ensure directory exists
CHROMA_DIR.mkdir(parents=True, exist_ok=True)
print(f"   ✅ Directory ready: {CHROMA_DIR}")

🗑️  Checking for existing ChromaDB v4 store …
   Found: c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\chromadb_store_v4  (0.000 GB)
   ⚠️  Unable to delete (file locks): [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\suhas\\OneDrive\\Desktop\\Capstone\\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\\main_folder\\chromadb_store_v4\\chroma.sqlite3'
   ℹ️  Will create new directory - collection delete in Step 5 will ensure clean state
   ✅ Directory ready: c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\chromadb_store_v4


---
## Step 1 — Load Input Artifacts

Load `X_knowledge_vectors_v4.npy` and `X_knowledge_meta_v4.parquet` produced by Phase 1.2 v4.

In [10]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 2 | Load Input Artifacts
# ─────────────────────────────────────────────────────────────────────────────
print("📦 Loading artifacts …")
t0 = time.time()

# ── Vectors ───────────────────────────────────────────────────────────────────
X_knowledge = np.load(str(VECTORS_PATH))
print(f"   ✅ X_knowledge       shape={X_knowledge.shape}  "
      f"dtype={X_knowledge.dtype}  "
      f"size={VECTORS_PATH.stat().st_size / 1e9:.3f} GB")

# ── Metadata ──────────────────────────────────────────────────────────────────
meta_knowledge = pd.read_parquet(str(META_PATH))
print(f"   ✅ meta_knowledge    shape={meta_knowledge.shape}  "
      f"size={META_PATH.stat().st_size / 1e6:.1f} MB")

# ── Alignment check ───────────────────────────────────────────────────────────
assert X_knowledge.shape[0] == len(meta_knowledge), (
    f"ALIGNMENT ERROR: vectors={X_knowledge.shape[0]} meta={len(meta_knowledge)}"
)
print(f"\n   ✅ Row alignment confirmed: {X_knowledge.shape[0]:,} rows")
print(f"   ✅ Loaded in {time.time()-t0:.1f}s")

# ── Preview ───────────────────────────────────────────────────────────────────
print(f"\n   Metadata columns : {list(meta_knowledge.columns)}")
print(f"\n   dtype overview   :")
print(meta_knowledge.dtypes.to_string())

print(f"\n📊 Full class distribution:")
total = len(meta_knowledge)
for lbl, cnt in meta_knowledge[LABEL_COL].value_counts().sort_index().items():
    name = LABEL_NAMES.get(int(lbl), str(lbl))
    print(f"   {int(lbl)} {name:<24} {cnt:>10,}  ({cnt/total*100:5.2f}%)")

print(f"\n   Sample metadata (first 3 rows):")
display(meta_knowledge.head(3))

📦 Loading artifacts …
   ✅ X_knowledge       shape=(9152557, 50)  dtype=float32  size=1.831 GB
   ✅ meta_knowledge    shape=(9152557, 8)  size=70.9 MB

   ✅ Row alignment confirmed: 9,152,557 rows
   ✅ Loaded in 8.2s

   Metadata columns : ['univ_label_multiclass', 'univ_label_str', 'univ_specific_attack', 'meta_src_ip', 'meta_dst_ip', 'meta_timestamp', 'dataset_source', 'univ_proto']

   dtype overview   :
univ_label_multiclass       int8
univ_label_str            object
univ_specific_attack      object
meta_src_ip               object
meta_dst_ip               object
meta_timestamp           float64
dataset_source            object
univ_proto                object

📊 Full class distribution:
   0 Normal                    1,018,128  (11.12%)
   1 Reconnaissance            2,526,173  (27.60%)
   2 Volumetric_Attack           307,229  ( 3.36%)
   3 C2_Botnet                   438,786  ( 4.79%)
   4 Exploit_and_Theft         4,862,241  (53.12%)

   Sample metadata (first 3 rows):


,univ_label_multiclass,univ_label_str,univ_specific_attack,meta_src_ip,meta_dst_ip,meta_timestamp,dataset_source,univ_proto
0,0,Normal,normal,3.122.49.24,192.168.1.152,1.554198e+09,None,tcp
1,0,Normal,normal,3.122.49.24,192.168.1.152,1.554220e+09,None,tcp
2,0,Normal,normal,3.122.49.24,192.168.1.152,1.554241e+09,None,tcp


---
## Step 2 — Stratified Split: 80% Ingest Set / 20% Live Simulation Set

`train_test_split` with `stratify=univ_label_multiclass` guarantees each class is
proportionally represented in both halves.

In [6]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 3 | Stratified Split
# ─────────────────────────────────────────────────────────────────────────────
print("✂️  Performing stratified 80/20 split …")
t0 = time.time()

# Generate indices for the full dataset
all_indices = np.arange(len(meta_knowledge))
labels      = meta_knowledge[LABEL_COL].values.astype(int)

ingest_idx, live_idx = train_test_split(
    all_indices,
    test_size    = TEST_SIZE,
    stratify     = labels,
    random_state = RANDOM_STATE,
    shuffle      = True,
)

# ── Slice vectors ─────────────────────────────────────────────────────────────
X_ingest = X_knowledge[ingest_idx]
X_live   = X_knowledge[live_idx]

# ── Slice metadata — preserve original DataFrame index for traceability ───────
meta_ingest = meta_knowledge.iloc[ingest_idx].reset_index(drop=True)
meta_live   = meta_knowledge.iloc[live_idx].reset_index(drop=True)

print(f"\n   Ingest Set  : {len(X_ingest):>10,} vectors  ({len(X_ingest)/len(X_knowledge)*100:.1f}%)")
print(f"   Live Set    : {len(X_live):>10,} vectors  ({len(X_live)/len(X_knowledge)*100:.1f}%)")
print(f"   Total       : {len(X_ingest)+len(X_live):>10,} == {len(X_knowledge):,}  ✅")
print(f"\n   Split completed in {time.time()-t0:.1f}s")

print(f"\n📊 Ingest Set class distribution:")
total_i = len(meta_ingest)
for lbl, cnt in meta_ingest[LABEL_COL].value_counts().sort_index().items():
    name = LABEL_NAMES.get(int(lbl), str(lbl))
    print(f"   {int(lbl)} {name:<24} {cnt:>10,}  ({cnt/total_i*100:5.2f}%)")

print(f"\n📊 Live Set class distribution:")
total_l = len(meta_live)
for lbl, cnt in meta_live[LABEL_COL].value_counts().sort_index().items():
    name = LABEL_NAMES.get(int(lbl), str(lbl))
    print(f"   {int(lbl)} {name:<24} {cnt:>10,}  ({cnt/total_l*100:5.2f}%)")

✂️  Performing stratified 80/20 split …

   Ingest Set  :  7,322,045 vectors  (80.0%)
   Live Set    :  1,830,512 vectors  (20.0%)
   Total       :  9,152,557 == 9,152,557  ✅

   Split completed in 6.2s

📊 Ingest Set class distribution:
   0 Normal                      814,502  (11.12%)
   1 Reconnaissance            2,020,938  (27.60%)
   2 Volumetric_Attack           245,783  ( 3.36%)
   3 C2_Botnet                   351,029  ( 4.79%)
   4 Exploit_and_Theft         3,889,793  (53.12%)

📊 Live Set class distribution:
   0 Normal                      203,626  (11.12%)
   1 Reconnaissance              505,235  (27.60%)
   2 Volumetric_Attack            61,446  ( 3.36%)
   3 C2_Botnet                    87,757  ( 4.79%)
   4 Exploit_and_Theft           972,448  (53.12%)


---
## Step 3 — Save Live Simulation Set

The 20% Live Set is written to disk **immediately** and is never touched again during ingestion.
It will serve as unseen "live traffic" for Phase 2 RAG retrieval evaluation.

In [7]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 4 | Save Live Simulation Set
# ─────────────────────────────────────────────────────────────────────────────
print("💾 Saving Live Simulation Set …")
t0 = time.time()

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# ── Vectors ───────────────────────────────────────────────────────────────────
np.save(str(LIVE_VECTORS_PATH), X_live)
live_vec_size = LIVE_VECTORS_PATH.stat().st_size
print(f"   ✅ {LIVE_VECTORS_PATH.name:<35}  "
      f"shape={X_live.shape}  "
      f"({live_vec_size / 1e6:,.1f} MB)")

# ── Metadata ──────────────────────────────────────────────────────────────────
meta_live.to_parquet(str(LIVE_META_PATH), index=False)
live_meta_size = LIVE_META_PATH.stat().st_size
print(f"   ✅ {LIVE_META_PATH.name:<35}  "
      f"shape={meta_live.shape}  "
      f"({live_meta_size / 1e6:,.1f} MB)")

print(f"\n   ✅ Live Set saved in {time.time()-t0:.1f}s")
print(f"\n   🔒 Live Set is now ON DISK — will not be referenced again during ingestion.")

💾 Saving Live Simulation Set …
   ✅ live_simulation_data_v4.npy          shape=(1830512, 50)  (366.1 MB)
   ✅ live_simulation_meta_v4.parquet      shape=(1830512, 8)  (29.2 MB)

   ✅ Live Set saved in 1.5s

   🔒 Live Set is now ON DISK — will not be referenced again during ingestion.


---
## Step 4 — Verify Split Integrity

Three hard assertions:
1. Ingest ∩ Live = ∅ (sets are disjoint)
2. |Ingest| + |Live| = |Total|
3. Both splits reproduce the expected class proportions

In [8]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 5 | Verify Split Integrity
# ─────────────────────────────────────────────────────────────────────────────
print("🔍 Verifying split integrity …")

ingest_set = set(ingest_idx.tolist())
live_set   = set(live_idx.tolist())

# ── Assertion 1: Disjoint sets ────────────────────────────────────────────────
overlap = ingest_set & live_set
assert len(overlap) == 0, f"CRITICAL: {len(overlap)} overlapping indices found!"
print(f"   ✅ Assertion 1 — Disjoint  : Ingest ∩ Live = ∅  (overlap={len(overlap)})")

# ── Assertion 2: Total count ──────────────────────────────────────────────────
assert len(ingest_set) + len(live_set) == len(X_knowledge), (
    f"Count mismatch: {len(ingest_set)}+{len(live_set)} != {len(X_knowledge)}"
)
print(f"   ✅ Assertion 2 — Totals    : {len(ingest_set):,} + {len(live_set):,} = {len(X_knowledge):,}")

# ── Assertion 3: Approximate class proportions ────────────────────────────────
print(f"\n   ✅ Assertion 3 — Stratification proportions:")
print(f"   {'Class':<28}  {'Base%':>7}  {'Ingest%':>8}  {'Live%':>7}  {'Max Δ':>7}")
print(f"   {'─'*28}  {'─'*7}  {'─'*8}  {'─'*7}  {'─'*7}")

max_delta = 0.0
for lbl in sorted(LABEL_NAMES.keys()):
    base_pct   = (labels == lbl).mean() * 100
    ingest_pct = (meta_ingest[LABEL_COL] == lbl).mean() * 100
    live_pct   = (meta_live[LABEL_COL]   == lbl).mean() * 100
    delta      = max(abs(ingest_pct - base_pct), abs(live_pct - base_pct))
    max_delta  = max(max_delta, delta)
    flag       = "✅" if delta < 0.5 else "⚠️ "
    name       = LABEL_NAMES[lbl]
    print(f"   {flag} {lbl} {name:<24}  {base_pct:>6.2f}%  {ingest_pct:>7.2f}%  {live_pct:>6.2f}%  {delta:>6.4f}")

assert max_delta < 2.0, f"Stratification drift too large: {max_delta:.4f}%"
print(f"\n   ✅ Max stratification drift: {max_delta:.4f}%  (< 2.0% threshold)")
print(f"\n✅ All integrity checks passed — safe to proceed with ingestion.")

🔍 Verifying split integrity …
   ✅ Assertion 1 — Disjoint  : Ingest ∩ Live = ∅  (overlap=0)
   ✅ Assertion 2 — Totals    : 7,322,045 + 1,830,512 = 9,152,557

   ✅ Assertion 3 — Stratification proportions:
   Class                           Base%   Ingest%    Live%    Max Δ
   ────────────────────────────  ───────  ────────  ───────  ───────
   ✅ 0 Normal                     11.12%    11.12%   11.12%  0.0000
   ✅ 1 Reconnaissance             27.60%    27.60%   27.60%  0.0000
   ✅ 2 Volumetric_Attack           3.36%     3.36%    3.36%  0.0000
   ✅ 3 C2_Botnet                   4.79%     4.79%    4.79%  0.0000
   ✅ 4 Exploit_and_Theft          53.12%    53.12%   53.12%  0.0000

   ✅ Max stratification drift: 0.0000%  (< 2.0% threshold)

✅ All integrity checks passed — safe to proceed with ingestion.


---
## Minimum Viable Standard (MVS) — Acceptance Criteria

**Context:** This v4 rebuild establishes a **new performance baseline** measured against **absolute thresholds**, not relative improvement over v2.

### Acceptance Thresholds (Phase 2 RAG Retrieval):

| Metric | Threshold | Rationale |
|--------|-----------|-----------|
| **Retrieval Latency** | < 50ms per query | Must support real-time IDS (industrial standard: sub-100ms) |
| **Normal Traffic Accuracy (Class 0)** | ≥ **85%** | Avoid false alarm fatigue; v2's 55% is categorized as **FAILURE** |
| **Attack Recall (Class 1-4)** | ≥ **95%** | Mission-critical: missed attacks = security breach |
| **Rare Variant Retention** | 100% | Verified in Phase 1.3 v4 (all 9 Exploit attack types present) |

### Key Changes from v2:

- **Normal Traffic Boost:** 422k → 1.02M vectors (11.1% of KB) — addresses v2's "Normal Traffic Starvation"
- **Volumetric Fix:** 13.88M (v3 explosion) → 307k vectors (3.4% of KB) — `src_ip` grouping restored
- **Total KB Size:** 9.15M vectors (vs 8.4M v2, 22.7M v3)

**Acceptance Decision:**  
If Phase 2 testing shows Normal Accuracy < 85% or Attack Recall < 95%, the v4 KB is rejected and Phase 1.2 must be re-engineered with adjusted reduction hyperparameters.

---
## Step 5 — Initialize ChromaDB Persistent Client & Collection

- **Store path:** `main_folder/chromadb_store_v4/`
- **Collection:** `rag_ids_knowledge_base`
- **Distance metric:** `cosine` (set via `hnsw:space`)

In [8]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 6 | Initialize ChromaDB Persistent Client & Collection
# ─────────────────────────────────────────────────────────────────────────────
print("🗄️  Initializing ChromaDB …")
print(f"   Store path : {CHROMA_DIR}")

client = chromadb.PersistentClient(path=str(CHROMA_DIR))

# ── Drop existing collection if re-running (idempotent) ───────────────────────
existing = [c.name for c in client.list_collections()]
if COLLECTION_NAME in existing:
    client.delete_collection(COLLECTION_NAME)
    print(f"   ⚠️  Existing collection '{COLLECTION_NAME}' deleted — fresh ingest.")

collection = client.create_collection(
    name     = COLLECTION_NAME,
    metadata = {"hnsw:space": "cosine"},
)

print(f"\n   ✅ Collection created : '{collection.name}'")
print(f"   Distance metric      : cosine  (hnsw:space)")
print(f"   Embedding dimension  : {X_ingest.shape[1]}")
print(f"   Current item count   : {collection.count()}")

# ── Collection info ───────────────────────────────────────────────────────────
print(f"\n   ChromaDB store contents:")
for fname in sorted(CHROMA_DIR.iterdir()):
    print(f"     {fname.name}")

🗄️  Initializing ChromaDB …
   Store path : c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\chromadb_store_v4
   ⚠️  Existing collection 'rag_ids_knowledge_base' deleted — fresh ingest.

   ✅ Collection created : 'rag_ids_knowledge_base'
   Distance metric      : cosine  (hnsw:space)
   Embedding dimension  : 50
   Current item count   : 0

   ChromaDB store contents:
     chroma.sqlite3


---
## Step 6 — Batch Ingestion into ChromaDB

Ingest the **80% Ingest Set** in batches of 40,000 vectors.  
Each document stores 5 metadata fields: `label`, `attack_type`, `src_ip`, `proto`, `dataset`.

In [9]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 7 | Batch Ingestion (The Heavy Lift)
# ─────────────────────────────────────────────────────────────────────────────
n_ingest     = len(X_ingest)
n_batches    = (n_ingest + BATCH_SIZE - 1) // BATCH_SIZE

print(f"🚀 Starting ChromaDB ingestion …")
print(f"   Total vectors : {n_ingest:>10,}")
print(f"   Batch size    : {BATCH_SIZE:>10,}")
print(f"   Num batches   : {n_batches:>10,}")
print()

# ── Pre-cache metadata columns as numpy arrays for fast slicing ───────────────
labels_arr   = meta_ingest[LABEL_COL].values.astype(int)
attack_arr   = meta_ingest["univ_specific_attack"].fillna("unknown").astype(str).values
src_ip_arr   = meta_ingest["meta_src_ip"].fillna("0.0.0.0").astype(str).values
proto_arr    = meta_ingest["univ_proto"].fillna("unknown").astype(str).values
dataset_arr  = meta_ingest["dataset_source"].fillna("unknown").astype(str).values

# ── Sanity check: none of the string fields have nulls left ───────────────────
for arr_name, arr in [("attack_arr", attack_arr), ("src_ip_arr", src_ip_arr),
                      ("proto_arr", proto_arr), ("dataset_arr", dataset_arr)]:
    nones = int(np.sum(arr == "None") + np.sum(arr == "nan"))
    if nones > 0:
        print(f"   ⚠️  {arr_name}: {nones} 'None'/'nan' strings found — will be kept as 'unknown'")

t0 = time.time()
total_ingested = 0

for batch_num in tqdm(range(n_batches), desc="Ingesting batches", unit="batch"):
    start = batch_num * BATCH_SIZE
    end   = min(start + BATCH_SIZE, n_ingest)

    batch_vectors = X_ingest[start:end].astype(np.float32)

    # ── IDs: unique string per vector ────────────────────────────────────────
    batch_ids = [f"v{ingest_idx[i]}" for i in range(start, end)]

    # ── Metadata dicts ────────────────────────────────────────────────────────
    batch_meta = [
        {
            "label":       int(labels_arr[i]),
            "attack_type": str(attack_arr[i]),
            "src_ip":      str(src_ip_arr[i]),
            "proto":       str(proto_arr[i]),
            "dataset":     str(dataset_arr[i]),
        }
        for i in range(start, end)
    ]

    # ── Convert vectors to Python lists (ChromaDB requirement) ───────────────
    batch_embeddings = batch_vectors.tolist()

    collection.add(
        embeddings = batch_embeddings,
        metadatas  = batch_meta,
        ids        = batch_ids,
    )
    total_ingested += (end - start)

elapsed = time.time() - t0
rate    = total_ingested / elapsed

print(f"\n✅ Ingestion complete!")
print(f"   Vectors ingested : {total_ingested:>10,}")
print(f"   Elapsed          : {elapsed/60:>7.1f} min  ({elapsed:.0f}s)")
print(f"   Throughput       : {rate:>10,.0f} vectors/s")
print(f"   Collection count : {collection.count():>10,}")

🚀 Starting ChromaDB ingestion …
   Total vectors :  7,322,045
   Batch size    :      5,000
   Num batches   :      1,465



Ingesting batches:   0%|          | 0/1465 [00:00<?, ?batch/s]


✅ Ingestion complete!
   Vectors ingested :  7,322,045
   Elapsed          :   165.6 min  (9939s)
   Throughput       :        737 vectors/s
   Collection count :  7,322,045


---
## Step 7 — Verification: Collection Count & Sanity-Check Query

1. Confirm the final ChromaDB item count matches the Ingest Set size.  
2. Pick **5 random vectors** from the **v4 Live Set** (never ingested) and query for their nearest neighbours.  
   Small cosine distances confirm the collection is correctly indexed and searchable.
3. Flag any distance > 0.1 for same-class matches as potential indexing errors.

In [14]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 8 | Verification: Collection Count & Sanity-Check Query
# ─────────────────────────────────────────────────────────────────────────────
print("=" * 65)
print("  VERIFICATION")
print("=" * 65)

# ── A: Collection count ────────────────────────────────────────────────────────
final_count = collection.count()
expected    = len(X_ingest)

print(f"\n  [A] Collection item count:")
print(f"      Expected : {expected:>10,}")
print(f"      Actual   : {final_count:>10,}")
assert final_count == expected, (
    f"COUNT MISMATCH: expected {expected}, got {final_count}"
)
print(f"      ✅ Count matches!")

# ── B: ChromaDB store size on disk ─────────────────────────────────────────────
total_bytes = sum(f.stat().st_size for f in CHROMA_DIR.rglob("*") if f.is_file())
print(f"\n  [B] ChromaDB store size: {total_bytes / 1e9:.3f} GB  ({CHROMA_DIR})")

# ── C: Sanity-check query with 5 random Live Set vectors ─────────────────────
print(f"\n  [C] Sanity-check query (5 random Live Set vectors → nearest neighbours):")

rng = np.random.default_rng(seed=7)
n_test_queries = 5
query_indices = rng.choice(len(X_live), size=n_test_queries, replace=False)

avg_latency = 0.0
all_distances = []

for i, query_idx in enumerate(query_indices):
    query_vec  = X_live[query_idx].astype(np.float32).tolist()
    query_meta = meta_live.iloc[query_idx]
    
    print(f"\n      Query {i+1}/{n_test_queries}:")
    print(f"        Index (in Live Set) : {query_idx:,}")
    print(f"        True label          : {int(query_meta[LABEL_COL])} "
          f"({LABEL_NAMES.get(int(query_meta[LABEL_COL]), '?')})")
    print(f"        Attack type         : {query_meta['univ_specific_attack']}")
    
    # ── Run query ─────────────────────────────────────────────────────────────
    t_q = time.time()
    results = collection.query(
        query_embeddings = [query_vec],
        n_results        = 1,
        include          = ["metadatas", "distances"],
    )
    latency_ms = (time.time() - t_q) * 1000
    avg_latency += latency_ms
    
    distance = results['distances'][0][0]
    all_distances.append(distance)
    nn_meta = results["metadatas"][0][0]
    nn_label = nn_meta.get("label", -1)
    label_match = nn_label == int(query_meta[LABEL_COL])
    
    print(f"        Distance            : {distance:.6f}  (cosine; 0=identical, 2=opposite)")
    print(f"        Latency             : {latency_ms:.1f} ms")
    print(f"        NN class            : {LABEL_NAMES.get(nn_label, '?')}")
    print(f"        Label match         : {'✅ YES' if label_match else '⚠️  NO (cross-class NN)'}")
    
    # ── Flag potential indexing errors ───────────────────────────────────────
    if distance > 0.1 and label_match:
        print(f"        ⚠️  HIGH DISTANCE for same-class match — potential indexing issue!")

avg_latency /= n_test_queries
avg_distance = sum(all_distances) / len(all_distances)

print(f"\n      ── Summary Statistics ──")
print(f"      Avg distance    : {avg_distance:.6f}")
print(f"      Avg latency     : {avg_latency:.1f} ms")
print(f"      Distance range  : [{min(all_distances):.6f}, {max(all_distances):.6f}]")

# ── Summary table ─────────────────────────────────────────────────────────────
print(f"\n{'='*65}")
print(f"  PHASE 1.4 SUMMARY")
print(f"{'='*65}")
print(f"  {'Total vectors':<35} : {len(X_knowledge):>12,}")
print(f"  {'Ingest Set (80%)':<35} : {len(X_ingest):>12,}")
print(f"  {'Live Set (20%)':<35} : {len(X_live):>12,}")
print(f"  {'ChromaDB collection count':<35} : {final_count:>12,}")
print(f"  {'ChromaDB store size':<35} : {total_bytes/1e9:>11.3f} GB")
print(f"  {'Avg sanity-check distance':<35} : {avg_distance:>12.6f}")
print(f"  {'Avg sanity-check latency':<35} : {avg_latency:>11.1f} ms")
print(f"{'='*65}")
print(f"\n✅ Phase 1.4 complete — ChromaDB knowledge base is ready for RAG!")

  VERIFICATION

  [A] Collection item count:
      Expected :  7,322,045
      Actual   :  7,322,045
      ✅ Count matches!

  [B] ChromaDB store size: 6.860 GB  (c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\chromadb_store_v4)

  [C] Sanity-check query (5 random Live Set vectors → nearest neighbours):

      Query 1/5:
        Index (in Live Set) : 1,144,242
        True label          : 4 (Exploit_and_Theft)
        Attack type         : xss
        Distance            : -0.000000  (cosine; 0=identical, 2=opposite)
        Latency             : 2.4 ms
        NN class            : Exploit_and_Theft
        Label match         : ✅ YES

      Query 2/5:
        Index (in Live Set) : 1,252,398
        True label          : 4 (Exploit_and_Theft)
        Attack type         : xss
        Distance            : 0.000000  (cosine; 0=identical, 2=opposite)
        Latency             : 1.5 ms
        NN class            : Exploit_and_The

In [15]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 9 | Git Commit
# ─────────────────────────────────────────────────────────────────────────────
import subprocess

REPO_ROOT = MAIN_DIR.parent

def run_git(args, cwd=REPO_ROOT):
    result = subprocess.run(
        ["git"] + args, cwd=str(cwd), capture_output=True, text=True
    )
    if result.returncode != 0:
        print(f"⚠️  git {' '.join(args[:2])} returned {result.returncode}")
        print(result.stderr.strip())
    return result.stdout.strip()

files_to_stage = [
    "main_folder/Phase_1/Phase_1_4_Knowledge_Ingestion.ipynb",
    "main_folder/data/processed/live_simulation_meta_v4.parquet",
    # live_simulation_data_v4.npy may be large — use LFS or skip if needed
]

# Track live_simulation_data_v4.npy only if under 100 MB (git limit)
live_npy_rel = "main_folder/data/processed/live_simulation_data_v4.npy"
if LIVE_VECTORS_PATH.exists() and LIVE_VECTORS_PATH.stat().st_size < 100 * 1e6:
    files_to_stage.append(live_npy_rel)
else:
    print(f"   ⏭️  Skipping large binary (>{100} MB): {LIVE_VECTORS_PATH.name}")

print("📋 Staging Phase 1.4 artifacts …")
for f in files_to_stage:
    abs_f = REPO_ROOT / f
    if abs_f.exists():
        run_git(["add", "-f", f])
        print(f"   ✅ staged  {f}")
    else:
        print(f"   ⏭️  skip (not found): {f}")

commit_msg = (
    f"Phase 1.4 v4: ChromaDB Knowledge Ingestion complete\n\n"
    f"- Built from Phase 1.2 v4 vectors (9.15M × 50)\n"
    f"- Stratified 80/20 split of {len(X_knowledge):,} vectors\n"
    f"- Ingest Set: {len(X_ingest):,} vectors → rag_ids_knowledge_base (cosine/hnsw)\n"
    f"- Live Set  : {len(X_live):,} vectors → live_simulation_data_v4.npy + meta_v4.parquet\n"
    f"- ChromaDB collection count: {final_count:,}\n"
    f"- Avg sanity-check distance : {avg_distance:.6f}\n"
    f"- Avg sanity-check latency  : {avg_latency:.1f} ms"
)

out = run_git(["commit", "-m", commit_msg])
print(f"\n{'='*65}")
print(out if out else "(nothing to commit / already up-to-date)")

log = run_git(["log", "--oneline", "-1"])

print(f"\n  Latest commit: {log}")
print(f"\n✅ Phase 1.4 committed!")

   ⏭️  Skipping large binary (>100 MB): live_simulation_data_v4.npy
📋 Staging Phase 1.4 artifacts …
   ✅ staged  main_folder/Phase_1/Phase_1_4_Knowledge_Ingestion.ipynb
   ✅ staged  main_folder/data/processed/live_simulation_meta_v4.parquet
⚠️  git commit -m returned 1


On branch main
Your branch is ahead of 'origin/main' by 13 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   main_folder/Phase_0/Phase_0_1_TON_IoT_Data_Understanding.ipynb
	modified:   main_folder/Phase_0/Phase_0_2_IoT23_Data_Understanding.ipynb
	modified:   main_folder/Phase_1/Phase_1_1_Universal_Schema_Alignment.ipynb
	deleted:    main_folder/Phase_1/Phase_1_2_Preprocessing_Normalization.ipynb
	modified:   main_folder/Phase_1/Phase_1_2_Vectorization_and_Smart_Reduction.ipynb
	modified:   main_folder/Phase_1/Phase_1_3_Deep_Validatio